# Eshelby Example

## Rotation functions

First we will define rotation functions for $Q_{ij}$ and $R_\sigma$

In [ ]:
import numpy as np

def qij_x(theta):
    """
    rotation tensor about x-axis by some theta
    """
    qij = np.array([[1,0,0],
                   [0,]])
    